In [41]:
# Standard library imports
import os
import sys
import re
import warnings
import random
import hashlib

# Data manipulation and analysis
import numpy as np
import pandas as pd

# Visualization libraries
import matplotlib.pyplot as plt
import seaborn as sns

# Machine learning and preprocessing
from sklearn.metrics import confusion_matrix, classification_report, precision_score
from sklearn.model_selection import RandomizedSearchCV, TimeSeriesSplit
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler  # Assuming you might need it

# Specific models and tools
from xgboost import XGBClassifier
import xgboost as xgb

# Encoding and feature selection
from category_encoders import TargetEncoder  # Fixed the import based on usage
from scipy.stats import randint, uniform

# Model persistence
from joblib import dump, load

# Miscellaneous settings
%matplotlib inline
warnings.filterwarnings('ignore')


In [42]:
comps = [
    'E0', 
    'E1', 
    
    #'E2', 'E3',    
    #'SC0', 'SC1',

    'D1', 
    #'D2',
    'F1', 
    #'F2',
    #
    'I1', 
    #'I2',
    'SP1', 
    #'SP2',
    #'B1',
    #'G1',
    #'N1',
    #'P1',
    #'T1',
]

seasons = [
    '2324', 
    '2223', '2122', '2021',
    '1920', 
    '1819', '1718', 
    #'1617',
    #'1516', '1415', '1314', '1213',
    #'1112', '1011', 
    #'0910', '0809',
    #'0708', '0607', '0506', '0405',
    #'0304', '0203', '0102', '0001',
]

countries = [
    "ARG", "AUT", "BRA", "CHN",
    "DNK", "FIN", "IRL", "JPN",
    "MEX", "NOR", "POL", "ROU",
    "RUS", "SWE", "SWZ", "USA",
]

fixtures = [
    "fixtures",
    #"new_league_fixtures"
]

content = "big5_7s"

In [43]:
# Load all filepaths into a list
matches_files = []

In [44]:
for season in seasons:    
    for comp in comps:  
        matches_files.append('data/scraped/%s/%s.csv' % (season, comp))
        #print()  # dummy line in case the line above is commented out

In [45]:
#for country in countries:    
    #matches_files.append('data/scraped/other/%s.csv' % (country))
    #print()  # dummy line in case the line above is commented out

In [46]:
#for fixture in fixtures:    
    #matches_files.append('data/scraped/fixtures/%s.csv' % (fixture))
    #print()  # dummy line in case the line above is commented out

In [47]:
matches_files

['data/scraped/2324/E0.csv',
 'data/scraped/2324/E1.csv',
 'data/scraped/2324/D1.csv',
 'data/scraped/2324/F1.csv',
 'data/scraped/2324/I1.csv',
 'data/scraped/2324/SP1.csv',
 'data/scraped/2223/E0.csv',
 'data/scraped/2223/E1.csv',
 'data/scraped/2223/D1.csv',
 'data/scraped/2223/F1.csv',
 'data/scraped/2223/I1.csv',
 'data/scraped/2223/SP1.csv',
 'data/scraped/2122/E0.csv',
 'data/scraped/2122/E1.csv',
 'data/scraped/2122/D1.csv',
 'data/scraped/2122/F1.csv',
 'data/scraped/2122/I1.csv',
 'data/scraped/2122/SP1.csv',
 'data/scraped/2021/E0.csv',
 'data/scraped/2021/E1.csv',
 'data/scraped/2021/D1.csv',
 'data/scraped/2021/F1.csv',
 'data/scraped/2021/I1.csv',
 'data/scraped/2021/SP1.csv',
 'data/scraped/1920/E0.csv',
 'data/scraped/1920/E1.csv',
 'data/scraped/1920/D1.csv',
 'data/scraped/1920/F1.csv',
 'data/scraped/1920/I1.csv',
 'data/scraped/1920/SP1.csv',
 'data/scraped/1819/E0.csv',
 'data/scraped/1819/E1.csv',
 'data/scraped/1819/D1.csv',
 'data/scraped/1819/F1.csv',
 'data/sc

In [48]:
# Load and concatenate matches data into a single DataFrame
df = pd.DataFrame()

for file in matches_files:

    try:
        df_temp = pd.read_csv(file)
        df = pd.concat([df, df_temp], ignore_index=True)
    except:
        # print an error message
        print(f'Error: {file} not found')

# print the amount of data loaded
print(f"Data loaded: {df.shape[0]} matches")

Data loaded: 16094 matches


In [49]:
# Rename columns if they exist
df.rename(columns={
    'Country': 'Div',
    'Home': 'HomeTeam',
    'Away': 'AwayTeam',
    'Res': 'FTR',

}, inplace=True)

In [50]:
# Remove all rows where 'Div' is not in the list of competitions
#df = df[df['Div'].isin(comps)].copy()

In [51]:
df.tail(20)

,Div,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,...,BbAv<2.5,BbAH,BbAHh,BbMxAHH,BbAvAHH,BbMxAHA,BbAvAHA,LBH,LBD,LBA
16074,SP1,12/05/18,NaN,Alaves,Ath Bilbao,3,1,H,1.0,0.0,...,1.64,17.0,-0.25,2.23,2.16,1.77,1.72,2.55,3.10,2.87
16075,SP1,12/05/18,NaN,Betis,Sevilla,2,2,D,1.0,0.0,...,2.35,18.0,0.00,1.99,1.92,1.99,1.93,2.55,3.50,2.55
16076,SP1,12/05/18,NaN,Eibar,Las Palmas,1,0,H,1.0,0.0,...,2.19,19.0,-1.25,1.93,1.88,2.02,1.97,1.40,4.60,8.00
16077,SP1,12/05/18,NaN,Getafe,Ath Madrid,0,1,A,0.0,1.0,...,1.46,18.0,0.00,1.95,1.90,2.10,1.96,2.70,2.90,2.80
16078,SP1,12/05/18,NaN,Girona,Valencia,0,1,A,0.0,0.0,...,2.13,18.0,0.00,1.93,1.88,2.03,1.97,2.50,3.40,2.62
16079,SP1,12/05/18,NaN,La Coruna,Villarreal,2,4,A,0.0,3.0,...,2.15,17.0,0.25,2.19,2.11,1.81,1.76,3.50,3.60,2.00
16080,SP1,12/05/18,NaN,Real Madrid,Celta,6,0,H,3.0,0.0,...,3.94,19.0,-1.50,1.96,1.91,2.00,1.95,1.36,5.50,7.50
16081,SP1,12/05/18,NaN,Sociedad,Leganes,3,2,H,2.0,1.0,...,2.04,19.0,-1.25,2.06,1.99,1.91,1.87,1.44,4.33,7.50
16082,SP1,13/05/18,NaN,Espanol,Malaga,4,1,H,3.0,1.0,...,1.74,19.0,-0.75,1.86,1.83,2.07,2.03,1.61,3.60,5.75
16083,SP1,13/05/18,NaN,Levante,Barcelona,5,4,H,2.0,1.0,...,3.23,18.0,1.50,2.11,2.06,1.86,1.81,9.00,5.50,1.30


In [52]:
# Check for duplicate column names
print(df.columns[df.columns.duplicated()])

Index([], dtype='object')


In [53]:
# Convert 'Div' to a categorical type, a numeric representation of the division
df['Div'] = df['Div'].astype('category').cat.codes

In [54]:
# Assuming 'teams' is a list of team names
teams = pd.concat([df['HomeTeam'], df['AwayTeam']]).dropna().unique()
teams.sort()

# Creating a dictionary from team names to an incremental index number
teams_dict = {team: index for index, team in enumerate(teams)}

In [55]:
# Save the dictionary to a file
with open(f'data/teams_dict_{content}.txt', 'w') as file:
    file.write(str(teams_dict))

In [56]:
# Create a unique list of HomeTeam and AwayTeam names combined, and add an index to each team
teams = pd.concat([df['HomeTeam'], df['AwayTeam']]).dropna().unique()

# Sort the teams alphabetically
teams.sort()

# Convert to an array of dictionaries
teams = [{'team': team, 'index': index} for index, team in enumerate(teams)]

df['Team_ID'] = df['HomeTeam'].map(teams_dict)
df['Opp_ID'] = df['AwayTeam'].map(teams_dict)

In [57]:
# if the column 'Referee' exists, convert it to a categorical type
if 'Referee' in df.columns:
    # Create a unique list of Referees, and add an index to each Referee
    referees = pd.concat([df['Referee']]).unique()

    # Convert to an array of dictionaries
    referees = [{'referee': referee, 'index': index} for index, referee in enumerate(referees)]

else:
    df['Referee'] = 0

In [58]:
# Correcting the creation of a unique list of Referees and adding an index to each Referee
referees = df['Referee'].unique()  # This should directly refer to the 'Referee' column

if len(referees) > 0:
    # Convert to a dictionary with referee names as keys and their indices as values
    referee_dict = {referee: index for index, referee in enumerate(referees)}

    # Now map the 'Referee' column to these indices
    df['Ref_ID'] = df['Referee'].map(referee_dict)
else:
    # If there are no referees, create a dummy column with all zeros
    df['Ref_ID'] = 0

In [59]:
import pandas as pd

def parse_date_to_int(date_str):
    # Split the date_str by the "/" character into day, month, year
    components = date_str.split('/')
    
    # If split was successful but not in expected format, try splitting by absence of separator for '%d%m%Y' or '%d%m%y'
    if len(components) == 1:
        if len(date_str) in [6, 8]:  # Length 6 for '%d%m%y', 8 for '%d%m%Y'
            day, month = int(date_str[:2]), int(date_str[2:4])
            year = int(date_str[4:])
        else:
            return 19000101  # Return default if format does not match expected
    else:
        day, month = int(components[0]), int(components[1])
        year = int(components[2])
    
    # Adjust the year if it was only 2 characters long
    if year < 100:
        year += 2000
    
    # Create a date variable by using the day, month, year integers
    # Note: Direct creation of date variable skipped to avoid unnecessary complexity,
    # directly formatting to YYYYMMDD integer format instead.
    date_int = int(f"{year:04d}{month:02d}{day:02d}")
    
    return date_int

# Assuming 'df' is your DataFrame and 'Date' is the column you want to convert
# First, ensure the Date column is in a datetime format if it's not already
df['Date'] = pd.to_datetime(df['Date'], errors='coerce', dayfirst=True)

# Apply the modified function
df['Date_temp'] = df['Date'].apply(lambda x: parse_date_to_int(x.strftime('%d/%m/%Y')) if pd.notnull(x) else 19000101)


In [60]:
df['Time'] = df['Time'].fillna('00:00').str.replace(':', '').astype(int)

In [61]:
df.columns = [re.sub(r'[<]', '_st_', str(col)) for col in df.columns]
df.columns = [re.sub(r'[>]', '_gt_', str(col)) for col in df.columns]

In [62]:
# Sort df by Date_temp and Time
df = df.sort_values(['Date_temp', 'Time'])

In [63]:
def history_vs_opponent_weighted(df, row, team_column):
    # Initialize the total weighted score
    weighted_score = 0
    opponent_column = 'Opp_ID'

    row_date_temp = row['Date'].year * 10000 + row['Date'].month * 100 + row['Date'].day

    
    # Filter the DataFrame for matches between the specified team and opponent from the same season, excluding the current match
    filtered_matches = df[(df[team_column] == row[team_column]) & 
                          (df[opponent_column] == row[opponent_column]) &
                          (df['Date_temp'] < row_date_temp)]
    
    recent_matches = filtered_matches.sort_values(by='Date', ascending=False).head(5)
    
    # Calculate weights - newer matches have higher weights
    weights = range(len(recent_matches), 0, -1)  # Descending list based on the number of matches
    
    # Calculate score based on the match result
    for match, weight in zip(recent_matches.itertuples(), weights):
        if getattr(match, 'FTR') == 'H' and getattr(match, team_column) == getattr(match, 'Team_ID') or \
           getattr(match, 'FTR') == 'A' and getattr(match, team_column) != getattr(match, 'Team_ID'):
            weighted_score += 3 * weight  # Team won
        elif getattr(match, 'FTR') == 'A':
            weighted_score += 1 * weight  # Draw
        
    # Normalize the weighted score by the sum of weights
    normalized_weighted_score = weighted_score / sum(weights) if weights else 0

    #print(f"Weighted score: {weighted_score}, Normalized weighted score: {normalized_weighted_score}")

    return normalized_weighted_score

In [64]:
df['team_hist_vs'] = [history_vs_opponent_weighted(df, row, 'Team_ID') for index, row in df.iterrows()]

In [65]:
# Apply the modified function to create new columns
df['team_hist_vs'] = df.apply(lambda x: history_vs_opponent_weighted(df, x, 'Team_ID'), axis=1)
df['opp_hist_vs'] = df.apply(lambda x: history_vs_opponent_weighted(df, x, 'Opp_ID'), axis=1)

In [66]:
import pandas as pd

# Function adapted for DataFrame application
def convert_odds(row):
    odds_win, odds_draw, odds_lose = row['AvgH'], row['AvgD'], row['AvgA']
    prob_win = 1 / odds_win
    prob_draw = 1 / odds_draw
    prob_lose = 1 / odds_lose
    prob_not_win = prob_draw + prob_lose
    return pd.Series([prob_win, prob_not_win], index=['probs_win', 'probs_not_win'])

# Apply the function and create new columns
#df[['probs_win', 'probs_not_win']] = df.apply(convert_odds, axis=1)

#df = df.drop(columns=['AvgH', 'AvgD', 'AvgA'])

In [67]:
def team_form(df, row, perspective):
    # Determine the team ID based on the perspective ('Team' or 'Opp')
    if perspective == 'Team':
        team_id = row['Team_ID']
    elif perspective == 'Opp':
        team_id = row['Opp_ID']
    else:
        raise ValueError("Perspective must be 'Team' or 'Opp'")
    
    # Get the current match date
    current_date = row['Date_temp']
    
    # Filter past matches for the team
    past_matches = df[((df['Team_ID'] == team_id) | (df['Opp_ID'] == team_id)) &
                      (df['Date_temp'] < current_date)].sort_values(by='Date_temp', ascending=False).head(5)
    
    # Initialize points
    points = 0
    
    # Weights for the matches (most recent match has the highest weight)
    weights = [5, 4, 3, 2, 1]
    
    # Calculate points with weights
    weighted_points_sum = 0
    total_weights = sum(weights[:len(past_matches)])  # Adjust the total weight in case of less than 5 matches
    
    for match, weight in zip(past_matches.itertuples(), weights):
        if (match.Team_ID == team_id and match.FTR == 'H') or (match.Opp_ID == team_id and match.FTR == 'A'):
            points += 3
        elif match.FTR == 'D':
            points += 1
        else:
            points += 0

        weighted_points_sum += points * weight
    
    if total_weights > 0:
        return weighted_points_sum / total_weights
    else:
        return 0  # Return 0 if no past matches found

In [68]:
df['team_form_team'] = [team_form(df, row, 'Team') for index, row in df.iterrows()]
df['team_form_opp'] = [team_form(df, row, 'Opp') for index, row in df.iterrows()]

In [69]:
# Applying the function to each row for 'Team'
df['team_form_team'] = df.apply(lambda row: team_form(df, row, 'Team'), axis=1)

# Applying the function to each row for 'Opp'
df['team_form_opp'] = df.apply(lambda row: team_form(df, row, 'Opp'), axis=1)

In [70]:
def rolling_avgs(df, row, perspective, home_column, away_column):
    # Determine the team ID based on the perspective ('Team' or 'Opp')
    if perspective == 'Team':
        team_id = row['Team_ID']
    elif perspective == 'Opp':
        team_id = row['Opp_ID']
    else:
        raise ValueError("Perspective must be 'Team' or 'Opp'")
    
    # Get the current match date
    current_date = row['Date_temp']
    
    # Filter past 5 matches for the team
    past_matches = df[((df['Team_ID'] == team_id) | (df['Opp_ID'] == team_id)) &
                      (df['Date_temp'] < current_date)].sort_values(by='Date_temp', ascending=False).head(5)
    
    # Weights for the matches (most recent match has the highest weight)
    weights = [5, 4, 3, 2, 1]
    values = []
    
    # Determine which column to use and collect the values
    for match in past_matches.itertuples():
        if match.Team_ID == team_id:
            values.append(getattr(match, home_column))  # Use home_column for home team
        else:
            values.append(getattr(match, away_column))  # Use away_column for away team
    
    # Calculate the weighted average of the values
    weighted_sum = sum(value * weight for value, weight in zip(values, weights))
    total_weights = sum(weights[:len(values)])  # Adjust total weight if there are less than 5 matches
    
    if total_weights > 0:
        return weighted_sum / total_weights
    else:
        return 0  # Return 0 if no past matches found


In [71]:
df['team_shots'] = df.apply(lambda row: rolling_avgs(df, row, 'Team', 'HS', 'AS'), axis=1)
df['opp_shots'] = df.apply(lambda row: rolling_avgs(df, row, 'Opp', 'HS', 'AS'), axis=1)

df['team_shots_target'] = df.apply(lambda row: rolling_avgs(df, row, 'Team', 'HST', 'AST'), axis=1)
df['opp_shots_target'] = df.apply(lambda row: rolling_avgs(df, row, 'Opp', 'HST', 'AST'), axis=1)

In [72]:
from datetime import timedelta

def avg_games_played(df, row, team_column):
    team = row[team_column]
    # Ensure current_match_date is a Timestamp for comparison
    current_match_date = pd.to_datetime(row['Date'], dayfirst=True)  # Assuming 'Date' format is 'dd/mm/yy'

    delta = 50
    start_date = current_match_date - timedelta(days=delta)

    # Ensure 'Date' column is in datetime format for comparison
    #df['Date_temp'] = pd.to_datetime(df['Date'], dayfirst=True)  # Convert 'Date' column to datetime if not already done

    # Filter the DataFrame for matches within the last 30 days
    if team_column == 'Team_ID':
        past_matches = df[((df[team_column] == team) | (df['Opp_ID'] == team)) &
                          (df['Date'] >= start_date) & (df['Date'] < current_match_date)]
    else:
        past_matches = df[((df['Team_ID'] == team) | (df[team_column] == team)) &
                          (df['Date'] >= start_date) & (df['Date'] < current_match_date)]

    # If no matches were played in the last 30 days
    if past_matches.empty:
        return 0

    # Calculate weights based on the recency of each match
    weights = (current_match_date - past_matches['Date']).dt.days
    weighted_count = sum(delta - weights + 1)  # '+ 1' to include the match day in the weight

    # Normalize weights to sum to 1 and calculate the weighted average
    total_weight = sum(delta - weights + 1)
    weighted_avg = weighted_count / total_weight

    return weighted_avg


In [73]:
# Apply the function for each team and opponent
df['team_avg_games'] = df.apply(lambda x: avg_games_played(df, x, 'Team_ID'), axis=1)
df['opp_avg_games'] = df.apply(lambda x: avg_games_played(df, x, 'Opp_ID'), axis=1)

In [74]:
# Calculate means only for numeric columns
numeric_cols = df.select_dtypes(include=[np.number]).columns
means = df[numeric_cols].mean()

# Fill missing values in numeric columns with their respective means
df[numeric_cols] = df[numeric_cols].fillna(means)

In [75]:
# Set the FTR to 'X' where the value is currently NaN
df['FTR'] = df['FTR'].fillna('X')

In [76]:
# Drop every row where 'FTR' is not 'H', 'D', or 'A', or 'X' (if future matches are included)
df = df[df['FTR'].isin(['H', 'D', 'A', 'X'])]

# FTR2 to store the FTR as '0', '1', or '2' for future reference
df['FTR2'] = df['FTR'].map({'H': 1, 'D': 0, 'A': 2, 'X': 0}).astype(int)

# Map 'H', 'D', and 'A' to 1, 0, and 0 respectively
df['FTR'] = df['FTR'].map({'H': 1, 'D': 0, 'A': 0, 'X': 0}).astype(int)

In [77]:
df = df[[
    
        'Div', 'Date_temp', 'Time', 'Team_ID', 'Opp_ID', 'Ref_ID', 'FTR', 'FTR2',
        'team_hist_vs', 
        #'opp_hist_vs',

        #'probs_win',         
        #'probs_not_win', 
        
        'team_form_team', 
        'team_form_opp',
         
        'team_shots', 'opp_shots',
        'team_shots_target', 'opp_shots_target',

        'team_avg_games', 'opp_avg_games',

        'AvgH', 'AvgD', 'AvgA'
         
         
         ]]

In [78]:
# Rename 'Date_temp' to 'Date'
df.rename(columns={'Date_temp': 'Date'}, inplace=True)

In [79]:
# Save the df to a CSV file
df.to_csv(f'data/processed/processed_data_{content}.csv', index=False)

In [80]:
import winsound
frequency = 400  # Set Frequency To 2500 Hertz
duration = 200  # Set Duration To 1000 ms == 1 second
winsound.Beep(frequency, duration)